<a href="https://colab.research.google.com/github/PhilSad/stable-diffusion-infinite-outpainting-video/blob/main/notebooks/colab_infinite_outpaint_generate_base_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Variable setup:

In [ ]:
# Choose a new SESSION_NAME
# will create a directory in your gdrive: MyDrive/stable-diffusion-infinite-outpaint/{SESSION_NAME}
# Choose a SESSION_NAME that you did not use before, otherwise the generated files will be replaced

SESSION_NAME = 'dummy' 
N_STEPS = '5'

Insert the prompts you want to use in a file called input.txt:

In [ ]:
%%file input.txt
A dog sitting in a bench park
A cat sitting in a bench park


Writing input.txt


In [ ]:
#@title Setup

import os
DRIVE_BASE_DIR = "/content/drive/MyDrive/stable-diffusion-infinite-outpaint"
BASE_IMAGE_DIR = f'{DRIVE_BASE_DIR}/{SESSION_NAME}/base_images'

if not os.path.exists(DRIVE_BASE_DIR):
  os.makedirs(DRIVE_BASE_DIR)

!git clone https://github.com/PhilSad/stable-diffusion-infinite-outpainting-video.git
%cd stable-diffusion-infinite-outpainting-video
!pip install -r requirements.txt


In [ ]:
!python infinite_outpaint.py --prompts_path ../input.txt --n_steps {N_STEPS} --downscale_factor 4 --save_folder {BASE_IMAGE_DIR}
